# Adding domains to the rddlrepository.

This follow-up example illustrates how to add a new context, domain and instance to the rddlrepository for later use.

First install and import the required packages:

In [12]:
%pip install --quiet --upgrade pip
%pip install --quiet git+https://github.com/pyrddlgym-project/pyRDDLGym.git
%pip install --quiet git+https://github.com/pyrddlgym-project/rddlrepository.git

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Import the required packages:

In [24]:
import pyRDDLGym
from pyRDDLGym.core.policy import RandomAgent
from rddlrepository.core.manager import RDDLRepoManager

Let's begin by adding a new context to the rddlrepository. We have to make sure it does not already exist:

In [14]:
manager = RDDLRepoManager(rebuild=True)
manager.register_context("mycontext")

Context <mycontext> was successfully registered in rddlrepository.


Next, we will create our RDDL domain specification. For illustration we will consider a simple example:

In [15]:
domain_text = """
domain simple_domain {

    pvariables {
        state : { state-fluent, int, default = 0 };
        action : { action-fluent, int, default = 0 };
    };
    
    cpfs {
        state' = state + action;
    };
    
    reward = pow[state' - 4, 2];
    
    action-preconditions {
        action >= -10 ^ action <= 10;
    };
}
"""

Let's register the domain in our newly-registered context, creating a description and assigning the default (graphical) visualizer:

In [16]:
manager.register_domain("SimpleDomain", "mycontext", domain_text, desc="a very simple additive domain", viz=None)

Domain <SimpleDomain> was successfully registered in rddlrepository with context <mycontext>.


Finally let's create at least one instance file for this domain:

In [17]:
instance_text = """
non-fluents nf_simple {
    domain = simple_domain;
}

instance simple_inst {
    domain = simple_domain;
    non-fluents = nf_simple;
    max-nondef-actions = pos-inf;
    horizon = 5;
    discount = 1.0;
}
"""

We will register the new instance as number 1 for the newly-registered domain:

In [20]:
problem_info = manager.get_problem("SimpleDomain_mycontext")
problem_info.register_instance("1", instance_text)

RDDLRepoInstanceDuplicationError: Instance <1> already exists in domain <SimpleDomain_mycontext>.

The moment of truth: let's check that everything works fine by loading the domain in pyRDDLGym:

In [23]:
RDDLRepoManager(rebuild=True)
env = pyRDDLGym.make("SimpleDomain_mycontext", "1")
agent = RandomAgent(action_space=env.action_space, num_actions=env.max_allowed_actions)
agent.evaluate(env, episodes=1, verbose=True)

initial state = 
     state = 0 
------------------------------------------------------------------------------------------------------------------------
step   = 0
action = 
     action = -8 
state  = 
     state = -8 
reward = 144.0
done   = False
------------------------------------------------------------------------------------------------------------------------
step   = 1
action = 
     action = -10 
state  = 
     state = -18 
reward = 484.0
done   = False
------------------------------------------------------------------------------------------------------------------------
step   = 2
action = 
     action = -5 
state  = 
     state = -23 
reward = 729.0
done   = False
------------------------------------------------------------------------------------------------------------------------
step   = 3
action = 
     action = -3 
state  = 
     state = -26 
reward = 900.0
done   = False
-----------------------------------------------------------------------------------------------

{'mean': 3218.0, 'median': 3218.0, 'min': 3218.0, 'max': 3218.0, 'std': 0.0}